# 0. Bibliotecas

In [ ]:
# mover os imports para cá

# 1. Datasets

In [ ]:
# carregar todos os datasets aqui

# 2. Pré-processamento dos dados

Uma vez que vamos trabalhar apenas com pontos no plano cartesiano, precisamos transformar nossos dados em uma **matriz bidimensional**. Para isso, utilizaremos a técnica **SVD** (Singular Value Decomposition) que reduz uma matriz *n*-dimensional em uma matriz *k*-dimensional, onde *k* é o número de fatores latentes. Para nosso trabalho, *k* é sempre 2.

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn import datasets
iris = datasets.load_iris() # apenas o iris tem duas classes linearmente separáveis
digits = datasets.load_digits()
wine = datasets.load_wine()
cancer = datasets.load_breast_cancer()

df = pd.DataFrame(iris.data, columns=iris.feature_names)
df['Target'] = pd.DataFrame(iris.target)


In [3]:
from sklearn.decomposition import TruncatedSVD

svd = TruncatedSVD(n_components=2, n_iter=4, random_state=42)
data_points = svd.fit_transform(df)

In [ ]:
df_new = pd.DataFrame(data_points, columns=['x','y'])
df_new['Target'] = pd.DataFrame(iris.target)
df_new

# 3. Envoltória Convexa

In [5]:
# classe para represemtar segmentos de reta no espaço
class Segment:
    def __init__(self, left, right):
        self.left = left
        self.right = right

In [6]:
# classe para representar pontos bidimensionais no espaço
class Ponto:
    def __init__(self, x, y):
        self.x = x
        self.y = y

In [7]:
#calculando o produto vetorial entre segmentos ab e ac, a = ancora
def produto_vetorial(ancora, b, c):
    return ((b.x-ancora.x)*(c.y-ancora.y) - (c.x-ancora.x)*(b.y-ancora.y))


In [8]:
# encontrando o ponto mais embaixo em relação a y
def mais_a_esquerda(pontos):
    min = 0
    for i in range(1,len(pontos)):
        if pontos[i].y < pontos[min].y:
            min = i
        elif pontos[i].y == pontos[min].y:
            if pontos[i].x < pontos[min].x: # em caso de empate, escolhemos o ponto mais a esquerda
                min = i
    return min

In [9]:
# funcao que recebe um vetor de pontos, e retorna lista com pontos da envoltoria convexa
def gift_wrapping(pontos):
    i_esquerda = mais_a_esquerda(pontos) # guarda indice do ponto mais a esquerda
    p = pontos[i_esquerda]

    prox = (i_esquerda + 1) % len(pontos)

    teste = 0

    envoltoria = [] #incializa envoltoria vazia
    envoltoria.append(p) # adiciona ponto mais à esquerda a envoltoria

    q = p
    while(True):
        # calcula o prox ponto da envoltoria
        while(teste != len(pontos)):
            det = produto_vetorial(q, pontos[prox], pontos[teste])
            if det < 0: # se for negativo, pontos[teste] tem o menor angulo polar,sendo o prox ponto da envoltoria
                prox = teste
            # atualiza teste
            teste = teste+1;

        # se tivermos retornados a ancora, loop acaba
        if p == pontos[prox]:
            break

        # adiciona ponto a envoltoria
        envoltoria.append(pontos[prox])

        # atualiza variáveis
        q = pontos[prox]
        prox = (prox + 1) % len(pontos)
        teste = 0

    return envoltoria

In [10]:
pontos = [Ponto(x, y) for x, y in data_points]
envoltoria = gift_wrapping(pontos)

In [ ]:
# Criando listas separadas para nomes e idades
Xs1 = [ponto.x for ponto in pontos]
Ys1 = [ponto.y for ponto in pontos]

Xs2 = [ponto.x for ponto in envoltoria]
Ys2 = [ponto.y for ponto in envoltoria]

# Adicionando o primeiro ponto da envoltória ao final para fechar a forma
Xs2.append(envoltoria[0].x)
Ys2.append(envoltoria[0].y)

# Plotando um gráfico de barras
plt.scatter(Xs1, Ys1, c=iris.target, s=20)
plt.plot(Xs2, Ys2, linestyle='-', marker='o', color='orange')
plt.xlabel('x')
plt.ylabel('x')
plt.title('Envoltória Convexa')
plt.show()

In [ ]:
# testando envoltoria com a matriz bidimensional
plt.clf()
plt.figure(figsize = (10, 6))
names = iris.target_names
label = (iris.target).astype(int)
colors = ['b','r','g']
for i in range(len(names)):
    bucket = df_new[df_new['Target'] == i]
    bucket = bucket.iloc[:,[0,1]].values
    bucket_points = [Ponto(x, y) for x, y in bucket]
    envoltoria[i] = gift_wrapping(bucket_points)
    Xs1 = [ponto.x for ponto in bucket_points]
    Ys1 = [ponto.y for ponto in bucket_points]
    plt.scatter(Xs1, Ys1, s=20)
    Xs2 = [ponto.x for ponto in envoltoria[i]]
    Ys2 = [ponto.y for ponto in envoltoria[i]]
    Xs2.append(envoltoria[i][0].x)
    Ys2.append(envoltoria[i][0].y)
    for j in envoltoria:
        plt.plot(Xs2, Ys2, linestyle='-', marker='o', color='orange')
plt.legend()
plt.show()

In [ ]:
#testando envoltoria com matriz inteira NAO EXECUTAR SE FOR TESTAR VARREDURA LINEAR
plt.clf()
plt.figure(figsize = (10, 6))
names = iris.target_names
label = (iris.target).astype(int)
colors = ['b','r','g']
plt.title('Petal Width vs Petal Length')
plt.xlabel(iris.feature_names[2])
plt.ylabel(iris.feature_names[3])
for i in range(len(names)):
    bucket = df[df['Target'] == i]
    bucket = bucket.iloc[:,[2,3]].values
    bucket_points = [Ponto(x, y) for x, y in bucket]
    envoltoria = gift_wrapping(bucket_points)
    Xs1 = [ponto.x for ponto in bucket_points]
    Ys1 = [ponto.y for ponto in bucket_points]
    plt.scatter(Xs1, Ys1, s=20)
    Xs2 = [ponto.x for ponto in envoltoria]
    Ys2 = [ponto.y for ponto in envoltoria]
    Xs2.append(envoltoria[0].x)
    Ys2.append(envoltoria[0].y)
    for j in envoltoria:
        plt.plot(Xs2, Ys2, linestyle='-', marker='o', color='orange')
plt.legend()
plt.show()

# 4. Varredura Linear


In [22]:
# testando com uma envoltoria apenas
line_segments = [Segment(envoltoria[0][i], envoltoria[0][i+1]) for i in range(len(envoltoria[0])-1)]
line_segments.append(Segment(envoltoria[0][-1], envoltoria[0][0]))

In [23]:
# A point in 2D plane
import bisect
# An event for sweep line algorithm
# An event has a point, the position
# of point (whether left or right) and
# index of point in the original input
# array of segments.
class Event:
    def __init__(self, x, y, isLeft, index):
        self.x = x
        self.y = y
        self.isLeft = isLeft
        self.index = index

    # This is for maintaining the order in set.
    def __lt__(self, e):
        if self.y == e.y:
            return self.x < e.x
        return self.y < e.y


In [24]:
# Given three collinear points p, q, r, the function checks if
# point q lies on line segment 'pr'
def onSegment(p, q, r):
    if (q.x <= max(p.x, r.x) and q.x >= min(p.x, r.x) and
        q.y <= max(p.y, r.y) and q.y >= min(p.y, r.y)):
        return True
    return False


In [25]:
# To find orientation of ordered triplet (p, q, r).
# The function returns following values
# 0 --> p, q and r are collinear
# 1 --> Clockwise
# 2 --> Counterclockwise

#é produto vetorial ? # milena
def orientation(p, q, r):
    val = (q.y - p.y) * (r.x - q.x) - (q.x - p.x) * (r.y - q.y)
    if val == 0:
        return 0  # collinear
    return 1 if val > 0 else 2  # clock or counterclock wise

In [26]:

# The main function that returns true if line segment 'p1q1'
# and 'p2q2' intersect.
def doIntersect(s1, s2):
    p1, q1, p2, q2 = s1.left, s1.right, s2.left, s2.right

    # Find the four orientations needed for general and
    # special cases
    o1 = orientation(p1, q1, p2)
    o2 = orientation(p1, q1, q2)
    o3 = orientation(p2, q2, p1)
    o4 = orientation(p2, q2, q1)

    # General case
    if o1 != o2 and o3 != o4:
        return True

    # Special Cases
    # p1, q1 and p2 are collinear and p2 lies on segment p1q1
    if o1 == 0 and onSegment(p1, p2, q1):
        return True

    # p1, q1 and q2 are collinear and q2 lies on segment p1q1
    if o2 == 0 and onSegment(p1, q2, q1):
        return True

    # p2, q2 and p1 are collinear and p1 lies on segment p2q2
    if o3 == 0 and onSegment(p2, p1, q2):
        return True

    # p2, q2 and q1 are collinear and q1 lies on segment p2q2
    if o4 == 0 and onSegment(p2, q1, q2):
        return True

    return False  # Doesn't fall in any of the above cases

In [27]:
# Find predecessor of iterator in s.
# index = sorted(s).index(it)
# return sorted(s)[index - 1] if index > 0 else None
# Find predecessor of iterator in s.
def pred(s, it):
    prev_elem = None
    for elem in s:
        if elem.x == it.x and elem.y == it.y and elem.isLeft:
            return prev_elem
        if elem.isLeft:
            prev_elem = elem
    return None

In [28]:
# Find successor of iterator in s.
# index = sorted(s).index(it)
# return sorted(s)[index + 1] if index < len(s) - 1 else None
# Find successor of iterator in s.
def succ(s, it):
    for elem in s:
        if elem.x == it.x and elem.y == it.y and elem.isLeft:
            next_elem = None
            for e in s:
                if e.x > it.x or (e.x == it.x and e.y > it.y):
                    if next_elem is None or (e.x < next_elem.x or (e.x == next_elem.x and e.y < next_elem.y)):
                        next_elem = e
            return next_elem
    return None


In [29]:
# Returns true if any two lines intersect.
def isIntersect(arr, n):
    mp = {}  # Dicionário para manter o registro dos pares para os quais a interseção já foi verificada

    # Adiciona todos os pontos a um vetor de eventos
    e = []
    for i in range(n):
        e.append(Event(arr[i].left.x, arr[i].left.y, True, i))
        e.append(Event(arr[i].right.x, arr[i].right.y, False, i))

    # Ordena todos os eventos de acordo com a coordenada x.
    e.sort(key=lambda x: x.x)

    #Para armazenar os segmentos ativos.
    s = set()

    ans = 0  # Inicializa o contador de interseções
    intersection_points = []  # Inicializa uma lista vazia para armazenar os pontos de intersecao

    for i in range(2 * n):
        curr = e[i]
        index = curr.index

        if curr.isLeft:
            next = succ(s, curr)  # Encontra o próximo segmento ativo
            prev = pred(s, curr)  # Encontra o segmento ativo anterior
            flag = False
            if next is not None and doIntersect(arr[next.index], arr[index]):
                intersection_points.append((arr[next.index].left, arr[index].left))
                # Se o próximo segmento ativo se intersecta com o segmento atual, adiciona o ponto de intersecao a lista
            if prev is not None and doIntersect(arr[prev.index], arr[index]):
                intersection_points.append((arr[prev.index].left, arr[index].left))
                # Se o segmento ativo anterior se intersecta com o segmento atual, adiciona o ponto de intersecao a lista
            if prev is not None and next is not None and next.index == prev.index:
                ans -= 1
                # Se o próximo e o anterior forem os mesmos segmentos, significa que o ponto de interseção foi contado duas vezes, então decrementa o contador de interseções

            s.add(curr)  # Adiciona o segmento atual ao conjunto de semgentos ativos
        else:
            it = None
            for elem in s:
                if elem.x == arr[index].left.x and elem.y == arr[index].left.y and elem.isLeft:
                    it = elem
                    break
                # Encontra o iterador correspondente ao segmento atual no conjunto de segmentos ativos
            if it is not None:
                next = succ(s, it)  # encontra o próximo segmento ativo
                prev = pred(s, it)  # encontra o segmento ativo anterior

                if next is not None and prev is not None:
                    if doIntersect(arr[prev.index], arr[next.index]):
                        intersection_points.append((arr[prev.index].left, arr[next.index].left))
                        # Se o segmento ativo anterior e o próximo se intersectam, adiciona o ponto de interseção a lista

                s.remove(it)  # remove o segmento atual do conjunto de segmentos ativos

    return intersection_points, ans  # Retorna a lista de pontos de interseção e a contagem de interseções

#n = len(line_segments)
#print(f"Number of intersection points: {isIntersect(line_segments, n)}")

n = len(line_segments)
intersection_points, count = isIntersect(line_segments, n)


In [30]:
for point_pair in intersection_points:
    print(f"Intersection Point 1: ({point_pair[0].x}, {point_pair[0].y}), Intersection Point 2: ({point_pair[1].x}, {point_pair[1].y})")
    # plt.scatter((point_pair[0].x,point_pair[0].y), (point_pair[1].x,point_pair[1].y) )
    # plt.plot((point_pair[0].x,point_pair[0].y), (point_pair[1].x,point_pair[1].y), linestyle='-', marker='o', color='orange')
    # plt.xlabel('x')
    # plt.ylabel('x')
    # plt.title('Envoltória Convexa')
    # plt.scatter(Xs1, Ys1)
    # plt.plot(Xs2, Ys2, linestyle='-', marker='o', color='orange')
    # plt.xlabel('x')
    # plt.ylabel('x')
    # plt.show()

#comentei porque isso plota os pontos e uma reta mostrando q eles se interceptam

Intersection Point 1: (4.917243400035973, 1.6795181589198684), Intersection Point 2: (4.917243400035973, 1.6795181589198684)
Intersection Point 1: (5.309743071875903, 2.611753667820411), Intersection Point 2: (5.309743071875903, 2.611753667820411)
Intersection Point 1: (6.452955398663373, 3.06463278165618), Intersection Point 2: (6.452955398663373, 3.06463278165618)
